In [1]:
import Bio
from Bio import Entrez, SeqIO, Seq

In [2]:
Entrez.email = "joshsteinbecker@gmail.com"

In [37]:
handle =       Entrez.efetch( 
                             db="nucleotide", 
                             id="NC_006577.2", 
                             rettype="gb", 
                             retmode="text") #IOWRAPPER

obj            = SeqIO.read(handle,'gb')

SEQ            = obj.seq
TAXO           = obj.annotations['taxonomy']
NAME           = obj.description
GENES          = [obj.features[x] for x in range(len(obj.features)) if obj.features[x].type == "gene"]
GNAMES         = [GENES[i].__dict__['qualifiers']['gene'][0] for i in range(len(GENES))]
SEQ, TAXO, NAME, GNAMES

(Seq('GAGTTTGAGCGATTGACGTTCGTACCGTCTATCAGCTTACGATCTCTTGTCAGA...TAC'),
 ['Viruses',
  'Riboviria',
  'Orthornavirae',
  'Pisuviricota',
  'Pisoniviricetes',
  'Nidovirales',
  'Cornidovirineae',
  'Coronaviridae',
  'Orthocoronavirinae',
  'Betacoronavirus',
  'Embecovirus'],
 'Human coronavirus HKU1, complete genome',
 ['ORF1ab', 'HE', 'S', 'orf4', 'E', 'M', 'N', 'N2'])

In [29]:
aa = seq.translate()
aa == aa.ungap()

/Users/joshsteinbecker/Library/Python/3.8/lib/python/site-packages/Bio/Seq.py:2979: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


True

In [31]:
defs = {
   "Good Suffix Rule" : 
      {"desc" : "If Suffix Matches then thats a good thing",
       "example": "ASS -- ASD"}
}

In [32]:
defs

{'Good Suffix Rule': {'desc': 'If Suffix Matches then thats a good thing',
  'example': 'ASS -- ASD'}}

In [34]:
import matplotlib.pyplot as plt
import pandas as pd
import sympy as sp
import random
import sklearn as skl

In [35]:
!pip3 install python-barcode

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.3/217.3 KB 1.8 MB/s eta 0:00:0000:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [91]:
import barcode as bc
from IPython.display import Image
html = """
<!DOCTYPE html>

<html>

<head>
   <style>
   h1   {color: powderblue;}
   p    {color: cyan;}
   </style>
</head>

<body>

   <h1>This is a heading</h1>
   
   <p>This is a paragraph.</p>

</body>

</html>"""


HTML(html)